In [1]:
import sys
import datetime
import time
from datetime import timedelta, date, datetime
from dateutil.relativedelta import relativedelta
import os
import urllib
import tarfile
import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from math import sin, cos, sqrt, atan2, radians

import glob

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

import geopandas as gpd

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
from matplotlib import cm
import haversine as hs
from shapely.geometry import MultiPoint, Point

from matplotlib.animation import FuncAnimation

In [2]:
ROx = pd.read_excel('../data_input/ROx_GCv12.xlsx', sheet_name='031924_ROx_list', header=None)
ROx = ROx.iloc[:, 0].tolist()
print(f'Total # of ROx species is {len(ROx)}.')
sorted(ROx)
ROx

Total # of ROx species is 34.


['A3O2',
 'ATO2',
 'B3O2',
 'BRO2',
 'DHPCARP',
 'DIBOO',
 'ETO2',
 'HC5OO',
 'HO2',
 'HPC52O2',
 'IEPOXOO',
 'INO2',
 'ISNOOA',
 'ISNOOB',
 'ISNOHOO',
 'ISOPNBO2',
 'ISOPNDO2',
 'KO2',
 'LIMO2',
 'MAOPO2',
 'MO2',
 'MOBAOO',
 'MRO2',
 'OH',
 'OTHRO2',
 'PIO2',
 'PO2',
 'PRN1',
 'R4N1',
 'R4O2',
 'RIO2',
 'TRO2',
 'VRO2',
 'XRO2']

### Select the RONO2 eqns and calculate stoichiometry based on Tropchem.eqn

In [3]:
file = open("/projectsp/f_xj103_1/jqshen/Code.12.7.1.PHCHO_Original/KPP/Tropchem/Tropchem.eqn","r")
lines = file.readlines()

In [4]:
def search_ROxplusNO_rxns(lines, ROx):
    def process_term(st):
        for i in range(len(st)):
            if 'A' <= st[i] <= 'Z':
                break
        if i == 0:
            return 1., st
        else:
            return float(st[:i]), st[i:]
    rnx_sum = []
    radical_change = []
    line_num = 391
    while line_num < len(lines):
        if "//" in lines[line_num] or not lines[line_num].strip():
            line_num += 1
            continue
        if ":" in lines[line_num]:
            line = lines[line_num]
            line, _ = line.split(':')
            line_num += 1
        elif ":" in lines[line_num+1]: 
            line = lines[line_num][:-1] + lines[line_num+1]
            line, _ = line.split(':')
            line_num += 2
        elif ":" in lines[line_num+2]:
            line = lines[line_num][:-1] + lines[line_num+1][:-1] + lines[line_num+2]
            line, _ = line.split(':')
            line_num += 3
        elif ":" in lines[line_num+3]:
            line = lines[line_num][:-1] + lines[line_num+1][:-1] + lines[line_num+2][:-1] + lines[line_num+3]
            line, _ = line.split(':')
            line_num += 4
        elif ":" in lines[line_num+4]:
            line = lines[line_num][:-1] + lines[line_num+1][:-1] + lines[line_num+2][:-1] + lines[line_num+3][:-1] + lines[line_num+4]
            line, _ = line.split(':')
            line_num += 5         
        elif ":" in lines[line_num+5]:
            line = (lines[line_num][:-1] + lines[line_num+1][:-1] + lines[line_num+2][:-1] + lines[line_num+3][:-1]
            + lines[line_num+4][:-1] + lines[line_num+5])
            line, _ = line.split(':')
            line_num += 6    
        else:
            raise NotImplementedError

        eqleft, eqright = line.split('=')
    
        for c in [' ', '{+M}', '\n']:
            eqleft = eqleft.replace(c, '')
            eqright = eqright.replace(c, '')
        eqleft = eqleft.split('+')
        eqright = eqright.split('+')

        flag = 0
        stoi = 0
        if "hv" in eqleft:
            continue
        spec_ = []
        for term in eqleft:
            _, spec = process_term(term)
            spec_.append(spec)
        if 'NO' in spec_:
            flag += 1
            spec_.remove('NO')
            if len(spec_) == 1 and any(item in ROx for item in spec_):
                flag += 1
        prod_ = []
        coef_ = [] 
        for term in eqright:
            coef, prod = process_term(term)
            coef_.append(coef)
            prod_.append(prod)
        ### list of RONO2
        RONO2 = pd.read_excel('../data_input/RONO2_GCv12.xlsx', sheet_name='040824', header=None)
        RONO2_list = RONO2.iloc[:, 0].tolist()
        if any(prod_i in RONO2_list for prod_i in prod_):
            flag += 1

        if flag == 3:
            rnx_sum.append(line)
            for i in range(len(prod_)):
                if prod_[i] in RONO2_list:
                    stoi = stoi + coef_[i]
            rci = stoi * (-1) 
            radical_change.append(f"{rci:.2f}")
    RONO2_eq = pd.DataFrame({'eq': rnx_sum, 'radical_change': radical_change})
    return RONO2_eq

In [5]:
RONO2_eq = search_ROxplusNO_rxns(lines, ROx)
pd.set_option('display.max_colwidth', None)
RONO2_eq

,eq,radical_change
0,MO2 + NO = MENO3,-1.00
1,ETO2 + NO = ETNO3,-1.00
2,A3O2 + NO = NPRNO3,-1.00
3,R4O2 + NO = R4N2,-1.00
4,KO2 + NO = 0.930NO2 + 0.930ALD2 + 0.930MCO3 + 0.070R4N2,-0.07
5,RIO2 + NO = 0.910NO2 + 0.820HO2 + 0.820CH2O + 0.476MVK + 0.344MACR + 0.058HC5 + 0.030DIBOO + 0.009ISOPND + 0.081ISOPNB,-0.09
6,VRO2 + NO = 0.880NO2 + 0.350HO2 + 0.350CH2O + 0.530MCO3 + 0.530GLYC + 0.350MGLY + 0.120MVKN,-0.12
7,MRO2 + NO = 0.850NO2 + 0.850HO2 + 0.122MGLY + 0.728HAC + 0.728CO + 0.122CH2O + 0.150MACRN,-0.15
8,B3O2 + NO = IPRNO3,-1.00
9,INO2 + NO = 1.300NO2 + 0.800HO2 + 0.700ISN1 + 0.230HC5 + 0.035MACR + 0.070CH2O + 0.035MVK,-0.70


### Get the rxn number of these reactions based on gckpp_Monitor.F90 (manually match)

In [6]:
RONO2_eq['rxn_num'] = np.nan

In [7]:
RONO2_eq.loc[RONO2_eq['eq'] == 'MO2 + NO = MENO3 ', 'rxn_num'] = 16
RONO2_eq.loc[RONO2_eq['eq'] == 'ETO2 + NO = ETNO3 ', 'rxn_num'] = 48
RONO2_eq.loc[RONO2_eq['eq'] == 'A3O2 + NO = NPRNO3 ', 'rxn_num'] = 53
RONO2_eq.loc[RONO2_eq['eq'] == 'R4O2 + NO = R4N2 ', 'rxn_num'] = 57
RONO2_eq.loc[RONO2_eq['eq'] == 'KO2 + NO = 0.930NO2 + 0.930ALD2 + 0.930MCO3 + 0.070R4N2 ', 'rxn_num'] = 60
RONO2_eq.loc[RONO2_eq['eq'] == 'RIO2 + NO = 0.910NO2 + 0.820HO2 + 0.820CH2O + 0.476MVK + 0.344MACR + 0.058HC5 + 0.030DIBOO + 0.009ISOPND + 0.081ISOPNB ', 'rxn_num'] = 61
RONO2_eq.loc[RONO2_eq['eq'] == 'VRO2 + NO = 0.880NO2 + 0.350HO2 + 0.350CH2O + 0.530MCO3 + 0.530GLYC + 0.350MGLY + 0.120MVKN ', 'rxn_num'] = 63
RONO2_eq.loc[RONO2_eq['eq'] == 'MRO2 + NO = 0.850NO2 + 0.850HO2 + 0.122MGLY + 0.728HAC + 0.728CO + 0.122CH2O + 0.150MACRN ', 'rxn_num'] = 64

In [8]:
RONO2_eq.loc[RONO2_eq['eq'] == 'B3O2 + NO = IPRNO3 ', 'rxn_num'] = 68
RONO2_eq.loc[RONO2_eq['eq'] == 'INO2 + NO = 1.300NO2 + 0.800HO2 + 0.700ISN1 + 0.230HC5 + 0.035MACR + 0.070CH2O + 0.035MVK ', 'rxn_num'] = 69
RONO2_eq.loc[RONO2_eq['eq'] == 'ISNOOA + NO = NO2 + 0.960PROPNN + CO + HO2 + 0.040ISN1OG ', 'rxn_num'] = 73
RONO2_eq.loc[RONO2_eq['eq'] == 'ISNOOB + NO = 0.900PROPNN + 0.940GLYX + 1.880NO2 + 0.040ISN1OG ', 'rxn_num'] = 77
RONO2_eq.loc[RONO2_eq['eq'] == 'ISNOHOO + NO = 0.894PROPNN + 0.934HO2 + 0.919GLYX + 0.0400ISN1OG ', 'rxn_num'] = 82
RONO2_eq.loc[RONO2_eq['eq'] == 'ISOPNDO2 + NO = 0.019MACRN + 0.057HCOOH + 0.270HAC + 0.210ETHLN + 0.150CH2O + 0.790NO2 + 0.300GLYC + 0.300PROPNN + 0.610HO2 + 0.270DHDN + 0.075MVKN + 0.057ISOPNDO2 ', 'rxn_num'] = 367
RONO2_eq.loc[RONO2_eq['eq'] == 'ISOPNBO2 + NO = 0.090GLYC + 0.090HAC + 0.700CH2O + 0.880NO2 + 0.440MACRN + 0.690HO2 + 0.260MVKN + 0.210DHDN ', 'rxn_num'] = 369
RONO2_eq.loc[RONO2_eq['eq'] == 'PIO2 + NO = 0.820HO2 + 0.820NO2 + 0.230CH2O + 0.430RCHO + 0.110ACET + 0.440MEK + 0.070HCOOH + 0.120MONITS + 0.060MONITU ', 'rxn_num'] = 441
RONO2_eq.loc[RONO2_eq['eq'] == 'LIMO2 + NO = 0.686HO2 + 0.780NO2 + 0.220MONITU + 0.289PRPE + 0.231CH2O + 0.491RCHO + 0.058HAC + 0.289MEK ', 'rxn_num'] = 453

### Correct radical change for reactions # 9-14

In [9]:
RONO2_eq.loc[RONO2_eq['eq'] == 'INO2 + NO = 1.300NO2 + 0.800HO2 + 0.700ISN1 + 0.230HC5 + 0.035MACR + 0.070CH2O + 0.035MVK ', 'radical_change'] = 0
RONO2_eq.loc[RONO2_eq['eq'] == 'ISNOOA + NO = NO2 + 0.960PROPNN + CO + HO2 + 0.040ISN1OG ', 'radical_change'] = 0
RONO2_eq.loc[RONO2_eq['eq'] == 'ISNOOB + NO = 0.900PROPNN + 0.940GLYX + 1.880NO2 + 0.040ISN1OG ', 'radical_change'] = 0
RONO2_eq.loc[RONO2_eq['eq'] == 'ISNOHOO + NO = 0.894PROPNN + 0.934HO2 + 0.919GLYX + 0.0400ISN1OG ', 'radical_change'] = -1.00
RONO2_eq.loc[RONO2_eq['eq'] == 'ISOPNDO2 + NO = 0.019MACRN + 0.057HCOOH + 0.270HAC + 0.210ETHLN + 0.150CH2O + 0.790NO2 + 0.300GLYC + 0.300PROPNN + 0.610HO2 + 0.270DHDN + 0.075MVKN + 0.057ISOPNDO2 ', 'radical_change'] = -0.21
RONO2_eq.loc[RONO2_eq['eq'] == 'ISOPNBO2 + NO = 0.090GLYC + 0.090HAC + 0.700CH2O + 0.880NO2 + 0.440MACRN + 0.690HO2 + 0.260MVKN + 0.210DHDN ', 'radical_change'] = -0.12

In [10]:
RONO2_eq['radical_change'] = pd.to_numeric(RONO2_eq['radical_change'])
RONO2_eq['radical_change'] = RONO2_eq['radical_change'].round(2)
RONO2_eq['rxn_num'] = RONO2_eq['rxn_num'].astype(int)
RONO2_eq_num = []
for i in range(len(RONO2_eq)):
    RONO2_eq_num.append(f"RxnRate_EQ{RONO2_eq.loc[i, 'rxn_num']:03d}")
RONO2_eq['RONO2_EQ_idx'] = RONO2_eq_num
RONO2_eq.drop(columns=['rxn_num'], inplace=True)
RONO2_eq

,eq,radical_change,RONO2_EQ_idx
0,MO2 + NO = MENO3,-1.00,RxnRate_EQ016
1,ETO2 + NO = ETNO3,-1.00,RxnRate_EQ048
2,A3O2 + NO = NPRNO3,-1.00,RxnRate_EQ053
3,R4O2 + NO = R4N2,-1.00,RxnRate_EQ057
4,KO2 + NO = 0.930NO2 + 0.930ALD2 + 0.930MCO3 + 0.070R4N2,-0.07,RxnRate_EQ060
5,RIO2 + NO = 0.910NO2 + 0.820HO2 + 0.820CH2O + 0.476MVK + 0.344MACR + 0.058HC5 + 0.030DIBOO + 0.009ISOPND + 0.081ISOPNB,-0.09,RxnRate_EQ061
6,VRO2 + NO = 0.880NO2 + 0.350HO2 + 0.350CH2O + 0.530MCO3 + 0.530GLYC + 0.350MGLY + 0.120MVKN,-0.12,RxnRate_EQ063
7,MRO2 + NO = 0.850NO2 + 0.850HO2 + 0.122MGLY + 0.728HAC + 0.728CO + 0.122CH2O + 0.150MACRN,-0.15,RxnRate_EQ064
8,B3O2 + NO = IPRNO3,-1.00,RxnRate_EQ068
9,INO2 + NO = 1.300NO2 + 0.800HO2 + 0.700ISN1 + 0.230HC5 + 0.035MACR + 0.070CH2O + 0.035MVK,0.00,RxnRate_EQ069


In [11]:
RONO2_eq = RONO2_eq[RONO2_eq['radical_change'] != 0]
RONO2_eq

,eq,radical_change,RONO2_EQ_idx
0,MO2 + NO = MENO3,-1.00,RxnRate_EQ016
1,ETO2 + NO = ETNO3,-1.00,RxnRate_EQ048
2,A3O2 + NO = NPRNO3,-1.00,RxnRate_EQ053
3,R4O2 + NO = R4N2,-1.00,RxnRate_EQ057
4,KO2 + NO = 0.930NO2 + 0.930ALD2 + 0.930MCO3 + 0.070R4N2,-0.07,RxnRate_EQ060
5,RIO2 + NO = 0.910NO2 + 0.820HO2 + 0.820CH2O + 0.476MVK + 0.344MACR + 0.058HC5 + 0.030DIBOO + 0.009ISOPND + 0.081ISOPNB,-0.09,RxnRate_EQ061
6,VRO2 + NO = 0.880NO2 + 0.350HO2 + 0.350CH2O + 0.530MCO3 + 0.530GLYC + 0.350MGLY + 0.120MVKN,-0.12,RxnRate_EQ063
7,MRO2 + NO = 0.850NO2 + 0.850HO2 + 0.122MGLY + 0.728HAC + 0.728CO + 0.122CH2O + 0.150MACRN,-0.15,RxnRate_EQ064
8,B3O2 + NO = IPRNO3,-1.00,RxnRate_EQ068
12,ISNOHOO + NO = 0.894PROPNN + 0.934HO2 + 0.919GLYX + 0.0400ISN1OG,-1.00,RxnRate_EQ082


In [12]:
RONO2_eq.to_csv('../data_output/RONO2_EQidx_RadicalChange.csv', index=False)